<a href="https://colab.research.google.com/github/nhwagle/code-match/blob/main/stream_paper_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a file we developed to scrap arxiv paper content in a respectful manner. A key difference is other versions ran out of memory, so here we download and process in a stream to throw away all unnecessary files before continuing downloads.

In [ ]:
import pandas as pd
import tarfile
import glob
import os
import time
from urllib import request
import asyncio
import time

In [ ]:
!pip install ratelimiter

In [ ]:
!pip install ipython ipykernel --upgrade

In [ ]:
from ratelimiter import RateLimiter

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/RSS/Links/latex-links.csv')
df.head()

,Unnamed: 0,paper,link,code,latex,id
0,0,Temporal coherence-based self-supervised learn...,http://arxiv.org/pdf/1806.06811v2.pdf,https://gitlab.com/nct_tso_public/pretrain_tc,http://arxiv.org/e-print/1806.06811,1806.06811
1,1,BinGAN: Learning Compact Binary Descriptors wi...,http://arxiv.org/pdf/1806.06778v5.pdf,https://github.com/maciejzieba/binGAN,http://arxiv.org/e-print/1806.06778,1806.06778
2,2,Multiscale Fisher's Independence Test for Mult...,https://arxiv.org/pdf/1806.06777v7.pdf,https://github.com/MaStatLab/multiFit,http://arxiv.org/e-print/1806.06777,1806.06777
3,3,POTs: Protective Optimization Technologies,https://arxiv.org/pdf/1806.02711v6.pdf,https://github.com/spring-epfl/pots,http://arxiv.org/e-print/1806.02711,1806.02711
4,4,Surface Networks,http://arxiv.org/pdf/1705.10819v2.pdf,https://github.com/jiangzhongshi/SurfaceNetworks,http://arxiv.org/e-print/1705.10819,1705.10819


In [ ]:
df[df.id == "2201.01852"]

,Unnamed: 0,paper,link,code,latex,id
51291,51291,Pruning deep neural networks generates a spars...,https://arxiv.org/pdf/2201.01852v1.pdf,https://github.com/oliviatessa/mothpruning,http://arxiv.org/e-print/2201.01852,2201.01852


In [ ]:
#pick up roughly where we left off...
# 1870 - 1967 in 5 min 23 seconds gives 3.3 seconds per paper..
# 1710.01218
df = df.iloc[51291:]

In [ ]:
df['latex'] = df[['latex']].applymap(lambda x: x[:7] + 'export.' + x[7:])

In [ ]:
df

,Unnamed: 0,paper,link,code,latex,id
51291,51291,Pruning deep neural networks generates a spars...,https://arxiv.org/pdf/2201.01852v1.pdf,https://github.com/oliviatessa/mothpruning,http://export.arxiv.org/e-print/2201.01852,2201.01852
51292,51292,SpineParseNet: Spine Parsing for Volumetric MR...,https://ieeexplore.ieee.org/document/9201093,https://github.com/pangshumao/SpineParseNet,http://export.arxiv.org/e-print/9201093,9201093
51293,51293,Benchmarking Data-driven Surrogate Simulators ...,https://openreview.net/pdf?id=-or413Lh_aF,https://github.com/ydeng-MLM/ML_MM_Benchmark,http://export.arxiv.org/e-print/pdf?id=-or413L...,pdf?id=-or413Lh_aF
51294,51294,DurLAR: A High-Fidelity 128-Channel LiDAR Data...,https://dro.dur.ac.uk/34293/1/34293.pdf?DDD4+D...,https://github.com/l1997i/DurLAR,http://export.arxiv.org/e-print/34293,34293
51295,51295,Fer2013 Recognition - ResNet18 With Tricks,https://github.com/LetheSec/Fer2013-Recognitio...,https://github.com/LetheSec/Fer2013-Facial-Emo...,http://export.arxiv.org/e-print/README.md,README.md
...,...,...,...,...,...,...
54050,54050,pyABC: Efficient and robust easy-to-use approx...,https://arxiv.org/pdf/2203.13043v1.pdf,https://github.com/icb-dcm/pyabc,http://export.arxiv.org/e-print/2203.13043,2203.13043
54051,54051,Event-Aware Multimodal Mobility Nowcasting,https://arxiv.org/pdf/2112.08443v1.pdf,https://github.com/underdoc-wang/east-net,http://export.arxiv.org/e-print/2112.08443,2112.08443
54052,54052,"""Good Robot! Now Watch This!"": Repurposing Rei...",https://proceedings.mlr.press/v164/hundt22a/hu...,https://github.com/jhu-lcsr/good_robot,http://export.arxiv.org/e-print/hundt22a,hundt22a
54053,54053,Interpretable EEG seizure prediction using a m...,https://www.nature.com/articles/s41598-022-083...,https://github.com/MauroSilvaPinto/Interpretab...,http://export.arxiv.org/e-print/s41598-022-083...,s41598-022-08322-w


In [ ]:
links = df['latex']
ids = df['id']
links

51291           http://export.arxiv.org/e-print/2201.01852
51292              http://export.arxiv.org/e-print/9201093
51293    http://export.arxiv.org/e-print/pdf?id=-or413L...
51294                http://export.arxiv.org/e-print/34293
51295            http://export.arxiv.org/e-print/README.md
                               ...                        
54050           http://export.arxiv.org/e-print/2203.13043
54051           http://export.arxiv.org/e-print/2112.08443
54052             http://export.arxiv.org/e-print/hundt22a
54053    http://export.arxiv.org/e-print/s41598-022-083...
54054    http://export.arxiv.org/e-print/S0303264721001994
Name: latex, Length: 2764, dtype: object

In [ ]:
base_path = "gdrive/MyDrive/RSS/"

In [ ]:

path = 'drive/MyDrive/RSS/Raw_Tex_Folder/'
most_recent_success=''


async def limited(until):
    duration = int(round(until - time.time()))
    print('Rate limited, sleeping for {:d} seconds'.format(duration))

async def coro():
    rate_limiter = RateLimiter(max_calls=4, period=1,callback=limited)
    
    for link, ID in zip(links, ids):
        
        print(link, ID)
        async with rate_limiter:
            try:
                request.urlretrieve(link, ID)
            except:
                print('Failed on paper', ID, ' at link:', link)
        try:
            curr_file = tarfile.open(ID)
            tex_tar_infos = [t_info for t_info in curr_file.getmembers() if t_info.name.find('.tex') != -1] 
            print([p.name for p in tex_tar_infos])
            print('')
            curr_file.extractall(base_path + 'Processed/' + ID + '/', members=tex_tar_infos)
            curr_file.close()
            os.remove(ID)
            most_recent_success = ID
        except:
            print(f"Failed parse on {ID}")


await coro()

Streaming output truncated to the last 5000 lines.
Failed parse on 2202.07123
http://export.arxiv.org/e-print/2202.07021 2202.07021
Rate limited, sleeping for 0 seconds
Failed on paper 2202.07021  at link: http://export.arxiv.org/e-print/2202.07021
Failed parse on 2202.07021
http://export.arxiv.org/e-print/2202.07474 2202.07474
Rate limited, sleeping for 0 seconds
Failed on paper 2202.07474  at link: http://export.arxiv.org/e-print/2202.07474
Failed parse on 2202.07474
http://export.arxiv.org/e-print/2202.07471 2202.07471
Failed on paper 2202.07471  at link: http://export.arxiv.org/e-print/2202.07471
Failed parse on 2202.07471
http://export.arxiv.org/e-print/2202.07421 2202.07421
Failed on paper 2202.07421  at link: http://export.arxiv.org/e-print/2202.07421
Failed parse on 2202.07421
http://export.arxiv.org/e-print/2202.07572 2202.07572
Rate limited, sleeping for 0 seconds
Failed on paper 2202.07572  at link: http://export.arxiv.org/e-print/2202.07572
Failed parse on 2202.07572
http:/

In [ ]:
most_recent_success

In [ ]:
os.remove(ID)

In [ ]:

# curr_file.close()

In [ ]:
import os 
os.listdir(base_path + '/Processed/')

In [ ]:
tex_tar_infos = [t_info for t_info in curr_file.getmembers() if t_info.name.find('.tex') != -1] 

In [ ]:
name = path[path.rfind('/')+1:]
  print(f'Processing item {name}, {index+1} of {len(paths)}.')


  tex_tar_infos = [t_info for t_info in curr_file.getmembers() if t_info.name.find('.tex') != -1] 
  print([p.name for p in tex_tar_infos])
  print('')
  
  curr_file.extractall(base_path + 'Processed/' + name + '/', members=tex_tar_infos)
  curr_file.close()